# Plotting the SIFT keypoint detector and feature descriptor error function
This document contains an experiment in which different keypoint detector and feature descriptor combinations are used to detect and uniquely describe common points of interest in image pairs. The goal of this experiment is to explore the effects that varying the values of the different input parameters of the detector/descriptors have on the frequency of inlier matches when computing a 3D relationship between images based upon epipolar geometry.

## Hypothesis

There exists an optimal set of descriptor/detector parameters that can minimize the number of outlier match pairs computer over a set of images.

## Experimental Design
1. Select three image sets:
    1. Nadir Facing Projective Camera.
    2. Nadir Facing Fisheye Camera.
    3. Oblique Camera.
3. Define a bound and a step size for the different parameters.
4. Load the images and initialize the Descriptor/Detector to the openCV initial values.
5. For each image in the image set compute the keypoints and descriptors.
6. For each pair of consecutive images, match corresponding keypoints.
7. Estimate the fundamental matrix between the image pair and compute the inliers and outliers for the image pair.
8. Sum the the number of inliers, outliers and total computed matches for the entire sequence.
9. Select one parameter and vary it by +/- the step size.
10. Repeat steps 5 - 9 while varying the selected parameter until it reaches the bounds.
11. Reset the parameter to its initial value.
12. Select a new parameter to vary and repeat 10.
13. Perform 3 - 12 for each image type.
    
    

## Experiment
To begin we import the necessary python packages including the custom built image processing modules

In [1]:
import image as im
import matching as mt
import imageUtility as ut
import transforms as tn
import numpy as np

### Select three image sets
The following three image datasets are used:
* __Nadir Facing Perspective Camera:__ Glacier images from the Sensefly eBee RTK
* __Nadir Facing Fisheye Camera:__ Images of the Quatre Fourches River in Wood Buffalo National Park, Alberta, Canada
* __Oblique Perspective Camera:__ Images of an office space

In [2]:
imageDir = {'glacier':{'dir':'/home/doopy/Documents/View3D/View3D_0_1/exp1/glacier/','ext':'.JPG'},
            'wbnp':{'dir':'/home/doopy/Documents/View3D/View3D_0_1/exp1/wbnp/','ext':'.JPG'},
            'desk':{'dir':'/home/doopy/Documents/View3D/View3D_0_1/exp1/glacier/','ext':'.png'}
           }

imgRange = range(1,10)

### Define a bound and a step size for the different parameters
In this experiment we will explore the following parameters:

| Parameter | Description | Initial Value | Step Size | Bounds |
| --- | --- | --- | --- | --- |
|# Features | _desc_ | 10000| 0 | - |
|# Octave Layers | _desc_ | 3| 1 | 1 - 5 |
|Contrast Threshold | _desc_ | 0.04| 0.01 | 0.01 - 0.08 |
|Edge Threshold | _desc_ | 10| 1 | 5 - 15 |
|Sigma | _desc_ | 1.6 | 0.1 | 1.0 - 2.0 | 


In [3]:
nOctaveLayers = 3
contrastThreshold = 0.04
edgeThreshold = 10
sigma = 1.6

### Initialize the Descriptor/Detector to the openCV initial values
Using the image class, create an image object for each image in the image set, and compute the keypoints and feature descriptors corresponding to the initial values

In [4]:
%%time

imgParams = {'scale':0.15,
          'kp':'sift',
          'nOctaveLayers':nOctaveLayers,
          'contrastThreshold':contrastThreshold,
          'edgeThreshold':edgeThreshold,
          'sigma':sigma
         }

imageSet = []

for i in imgRange:
    
    imageSet.append(im.image(imageDir['glacier']['dir'] + str(i) + imageDir['glacier']['ext'],imgParams))
    

CPU times: user 12.3 s, sys: 204 ms, total: 12.5 s
Wall time: 10.9 s


### For each consecutive pair of images match corresponding keypoints and compute the fundamental matrix
Using the matching class compute the first-last approximate nearest neighbour algorithm in order to match keypoints in consecutive pairs within the image set.
The transformation class is then used to compute the fundamental matrix between the two images.

In [5]:

params = {'nOctaveLayers':{'step':1,'iv':1,'bdy':6},
        'contrastThreshold':{'step':0.01,'iv':0.01,'bdy':0.09},
        'edgeThreshold':{'step':1,'iv':5,'bdy':16},
        'sigma':{'step':0.1,'iv':1.0,'bdy':2.1},
         }


results = {'nOctaveLayers':{'value':[],'inlierCount':[],'outlierCount':[]},
        'contrastThreshold':{'value':[],'inlierCount':[],'outlierCount':[]},
        'edgeThreshold':{'value':[],'inlierCount':[],'outlierCount':[]},
        'sigma':{'value':[],'inlierCount':[],'outlierCount':[]},
         }

for parameter in params.keys():
    
    print '*********** ' + parameter + ' ************'
    
    imgParams = {'scale':0.15,
          'kp':'sift',
          'nOctaveLayers':nOctaveLayers,
          'contrastThreshold':contrastThreshold,
          'edgeThreshold':edgeThreshold,
          'sigma':sigma
         }

    imgParams[parameter] = params[parameter]['iv']
    
    for j in np.arange(imgParams[parameter],params[parameter]['bdy'],params[parameter]['step']):
        
        print '*********** ' + str(j) + ' ***********'
        
        for img in imageSet:
            
            img.computeKP(imgParams)
            
        img1 = imageSet[0]
        inlierSum = 0
        outlierSum = 0

        for img2 in imageSet[1:]:

            F = tn.fundamental(img1,img2,imgParams)

            inlierSum += F.inlierCount()
            outlierSum += F.outlierCount()

            img1 = img2   
        
        results[parameter]['value'].append(imgParams[parameter])
        results[parameter]['inlierCount'].append(inlierSum)
        results[parameter]['outlierCount'].append(outlierSum)
        
        imgParams[parameter] = j

*********** sigma ************
*********** 1.0 ***********
*********** 1.1 ***********
*********** 1.2 ***********
*********** 1.3 ***********
*********** 1.4 ***********
*********** 1.5 ***********
*********** 1.6 ***********
*********** 1.7 ***********
*********** 1.8 ***********
*********** 1.9 ***********
*********** 2.0 ***********
*********** nOctaveLayers ************
*********** 1 ***********
*********** 2 ***********
*********** 3 ***********
*********** 4 ***********
*********** 5 ***********
*********** contrastThreshold ************
*********** 0.01 ***********
*********** 0.02 ***********
*********** 0.03 ***********
*********** 0.04 ***********
*********** 0.05 ***********
*********** 0.06 ***********
*********** 0.07 ***********
*********** 0.08 ***********
*********** edgeThreshold ************
*********** 5 ***********
*********** 6 ***********
*********** 7 ***********
*********** 8 ***********
*********** 9 ***********
*********** 10 ***********
*********** 11 ******

In [6]:
results

{'contrastThreshold': {'inlierCount': [23466,
   23466,
   23210,
   22534,
   21361,
   20612,
   19261,
   16933],
  'outlierCount': [916, 916, 404, 517, 897, 441, 217, 683],
  'value': [0.01,
   0.01,
   0.02,
   0.029999999999999999,
   0.040000000000000001,
   0.050000000000000003,
   0.060000000000000005,
   0.069999999999999993]},
 'edgeThreshold': {'inlierCount': [18658,
   18658,
   19762,
   20555,
   20732,
   21330,
   21361,
   21826,
   21858,
   21786,
   22438],
  'outlierCount': [593, 593, 683, 647, 921, 699, 897, 579, 666, 836, 253],
  'value': [5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]},
 'nOctaveLayers': {'inlierCount': [8302, 8302, 16661, 21361, 24726],
  'outlierCount': [204, 204, 151, 897, 771],
  'value': [1, 1, 2, 3, 4]},
 'sigma': {'inlierCount': [19190,
   19190,
   20898,
   22881,
   24331,
   24585,
   23955,
   21361,
   19733,
   17809,
   16160],
  'outlierCount': [781, 781, 579, 1070, 634, 874, 429, 897, 538, 545, 586],
  'value': [1.0,
   1.0,
   1.100000